# FourSquare Data by Neighborhood

## Import Libraries

In [1]:
import pandas as pd
import numpy as np
import geopandas as gpd
import json
import requests
from pandas.io.json import json_normalize

import shapefile
from shapely.geometry import Point
from shapely.geometry.polygon import Polygon

from sklearn.cluster import KMeans

In [2]:
filename = 'Neighborhood_Demographics.csv'
location_data = pd.read_csv(filename)

In [3]:
len(location_data)

2575

In [4]:
location_data.tail()

,location,latitude,longitude,address,street,city,county,state,zipcode,male,female,veterans,owners,renters,median_home,families,median_income,median_age,hispanic,non_hispanic
2570,"Pittman (Altoona, FL)",28.996888,-81.644155,"19005 Lake Rd, Altoona, FL 32702",Lake,Altoona,Lake County,FL,32702,0.389,0.611,0.152,0.568,0.432,112500,0.512,37950,61.2,0.068,0.932
2571,"Lake Kathryn (Paisley, FL)",29.003815,-81.492781,"44302 Kentucky St, Paisley, FL 32767",Kentucky,Paisley,Lake County,FL,32767,0.532,0.468,0.169,0.866,0.134,122800,0.859,61234,51.9,0.045,0.955
2572,"Pine Island Center (Bokeelia, FL)",26.634247,-82.128795,"6251 Alcorn St, Bokeelia, FL 33922",Alcorn,Bokeelia,Lee County,FL,33922,0.481,0.519,0.091,0.694,0.306,145600,0.561,33250,59.6,0.008,0.992
2573,"Pineland (Bokeelia, FL)",26.665114,-82.145258,"7252 Hibiscus Ave, Bokeelia, FL 33922",Hibiscus,Bokeelia,Lee County,FL,33922,0.512,0.488,0.192,0.880,0.120,172600,0.647,42375,66.2,0.230,0.770
2574,"Useppa Island (Bokeelia, FL)",26.661235,-82.213334,"328 Useppa Is, Captiva, FL 33924",Useppa,Captiva,Lee County,FL,33924,0.355,0.645,0.323,1.000,0.000,327300,1.000,69000,66.6,0.000,1.000


## FourSquare API Calls

In [9]:
LIMIT = 100
radius = 500

In [10]:
with open ('FourSquare_API.txt', 'r') as f:
    
    CLIENT_ID = f.readline().strip()
    CLIENT_SECRET = f.readline().strip()
    VERSION = f.readline().strip()

In [11]:
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    latitude, 
    longitude, 
    radius, 
    LIMIT)

In [12]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5df2718e14a126001b834225'},
 'response': {'warning': {'text': "There aren't a lot of results near you. Try something more general, reset your filters, or expand the search area."},
  'headerLocation': 'Frostproof',
  'headerFullLocation': 'Frostproof',
  'headerLocationGranularity': 'city',
  'totalResults': 1,
  'suggestedBounds': {'ne': {'lat': 27.76363400450001,
    'lng': -81.45616824627851},
   'sw': {'lat': 27.754633995499997, 'lng': -81.46631975372148}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4d5412e3c8d0f04dcddcbdd4',
       'name': 'Big Bay Loop Trail',
       'location': {'address': 'County Road 630',
        'lat': 27.76005302575479,
        'lng': -81.46529912610518,
        'labeledLatLngs': [{'label': 'display',

## Extract Venue and Categorize

In [13]:
def get_category_type(row):
    
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [14]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues)

filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Big Bay Loop Trail,Trail,27.760053,-81.465299


In [15]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

1 venues were returned by Foursquare.


## Explore Neighborhoods

In [16]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    
    for name, lat, lng in zip(names, latitudes, longitudes):
            
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        try:
            
            results = requests.get(url).json()["response"]['groups'][0]['items']
        
            venues_list.append([(
                name, 
                lat, 
                lng, 
                v['venue']['name'], 
                v['venue']['location']['lat'], 
                v['venue']['location']['lng'],  
                v['venue']['categories'][0]['name']) for v in results])

        except:
            
            pass
        
    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                             'Neighborhood Latitude', 
                             'Neighborhood Longitude', 
                             'Venue', 
                             'Venue Latitude', 
                             'Venue Longitude', 
                             'Venue Category']
    
    return(nearby_venues)

In [19]:
%%time

candidates_venues = getNearbyVenues(names=location_data['location'],
                                    latitudes=location_data['latitude'],
                                    longitudes=location_data['longitude']
                                    )

Wall time: 16min 7s


In [21]:
print(candidates_venues.shape)
candidates_venues.head()

(15974, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"CoreLife (833 North Krocks Rd, Suite 101)",40.563454,-75.562741,CoreLife Eatery,40.565079,-75.562347,Gluten-free Restaurant
1,"CoreLife (833 North Krocks Rd, Suite 101)",40.563454,-75.562741,Anthony's Coal Fired Pizza,40.564532,-75.565272,Pizza Place
2,"CoreLife (833 North Krocks Rd, Suite 101)",40.563454,-75.562741,Nordstrom Rack,40.563754,-75.566477,Discount Store
3,"CoreLife (833 North Krocks Rd, Suite 101)",40.563454,-75.562741,The Shelby,40.565773,-75.562251,New American Restaurant
4,"CoreLife (833 North Krocks Rd, Suite 101)",40.563454,-75.562741,Target,40.565977,-75.560674,Big Box Store


In [22]:
candidates_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
"29th and Chase (Jacksonville, FL)",5,5,5,5,5,5
"33rd St. Industrial (Orlando, FL)",10,10,10,10,10,10
"441 Corridor (Hollywood, FL)",4,4,4,4,4,4
"5 Points (Sarasota, FL)",68,68,68,68,68,68
"Aberdeen (Wesley Chapel, FL)",5,5,5,5,5,5
...,...,...,...,...,...,...
"Yellow Banks Grove (Largo, FL)",9,9,9,9,9,9
"Yukon (Jacksonville, FL)",3,3,3,3,3,3
"Zephyrhills North (Zephyrhills, FL)",1,1,1,1,1,1


In [23]:
print('There are {} unique categories.'.format(len(candidates_venues['Venue Category'].unique())))

There are 467 unique categories.


## Analyze Each Neighborhood

In [24]:
candidates_onehot = pd.get_dummies(candidates_venues[['Venue Category']], prefix="", prefix_sep="")

candidates_onehot['Neighborhood'] = candidates_venues['Neighborhood'] 

fixed_columns = [candidates_onehot.columns[-1]] + list(candidates_onehot.columns[:-1])
candidates_onehot = candidates_onehot[fixed_columns]

candidates_onehot.head()

,Zoo Exhibit,ATM,Accessories Store,Adult Boutique,Advertising Agency,Airport,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,...,Waterfront,Weight Loss Center,Whisky Bar,Wine Bar,Wine Shop,Winery,Wings Joint,Women's Store,Yoga Studio,Zoo
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [25]:
candidates_grouped = candidates_onehot.groupby('Neighborhood').mean().reset_index()
candidates_grouped.head()

,Neighborhood,Zoo Exhibit,ATM,Accessories Store,Adult Boutique,Advertising Agency,Airport,Airport Lounge,Airport Service,Airport Terminal,...,Waterfront,Weight Loss Center,Whisky Bar,Wine Bar,Wine Shop,Winery,Wings Joint,Women's Store,Yoga Studio,Zoo
0,"29th and Chase (Jacksonville, FL)",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0
1,"33rd St. Industrial (Orlando, FL)",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0
2,"441 Corridor (Hollywood, FL)",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0
3,"5 Points (Sarasota, FL)",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.014706,0.0,0.0,0.0,0.014706,0.0,0.0
4,"Aberdeen (Wesley Chapel, FL)",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.2,0.0


## Find Top Venues which Characterize a Neighborhood

In [37]:
num_top_venues = 5

for hood in candidates_grouped['Neighborhood']:
    
    print("----"+hood+"----")
    temp = candidates_grouped[candidates_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    
    print('\n')

----29th and Chase (Jacksonville, FL)----
                venue  freq
0      Sandwich Place   0.4
1      Discount Store   0.2
2  Chinese Restaurant   0.2
3           BBQ Joint   0.2
4         Zoo Exhibit   0.0


----33rd St. Industrial (Orlando, FL)----
                        venue  freq
0  Construction & Landscaping   0.2
1                     Brewery   0.1
2              Gymnastics Gym   0.1
3            Toy / Game Store   0.1
4                      Bakery   0.1


----441 Corridor (Hollywood, FL)----
                 venue  freq
0  Rental Car Location  0.50
1          Gas Station  0.25
2           Smoke Shop  0.25
3          Zoo Exhibit  0.00
4         Outlet Store  0.00


----5 Points (Sarasota, FL)----
                 venue  freq
0   Italian Restaurant  0.07
1  American Restaurant  0.07
2       Breakfast Spot  0.06
3          Coffee Shop  0.06
4     Sushi Restaurant  0.06


----Aberdeen (Wesley Chapel, FL)----
                        venue  freq
0                 Yoga Studio   0.

4     Convenience Store  0.11


----Arlington Hills (Jacksonville, FL)----
                        venue  freq
0              Baseball Field  0.33
1                         Gym  0.17
2                      Lounge  0.17
3  Construction & Landscaping  0.17
4                  Hobby Shop  0.17


----Arlington Park (Sarasota, FL)----
              venue  freq
0  Business Service  0.25
1              Park  0.25
2      Tennis Court  0.25
3        Playground  0.25
4       Zoo Exhibit  0.00


----Arlingwood (Jacksonville, FL)----
               venue  freq
0  Convenience Store  0.33
1               Park  0.33
2        Zoo Exhibit  0.00
3       Outlet Store  0.00
4        Pet Service  0.00


----Armory Gardens (Tampa, FL)----
                  venue  freq
0        Sandwich Place  0.18
1             BBQ Joint  0.09
2  Gym / Fitness Center  0.09
3    Chinese Restaurant  0.09
4    Mexican Restaurant  0.09


----Arrowhead (Jacksonville, FL)----
                venue  freq
0  Chinese Restaurant  0.25

4  Sporting Goods Shop   0.2


----Bartlett Park (Saint Petersburg, FL)----
                  venue  freq
0  Fast Food Restaurant   0.4
1    Mexican Restaurant   0.1
2            Donut Shop   0.1
3           Coffee Shop   0.1
4       Harbor / Marina   0.1


----Bass Creek (Delray Beach, FL)----
               venue  freq
0                Gym  0.25
1  Martial Arts Dojo  0.25
2     Ice Cream Shop  0.25
3       Dessert Shop  0.25
4  Paella Restaurant  0.00


----Bay Aristocrat Village (Clearwater, FL)----
             venue  freq
0              Gym  0.22
1             Pool  0.22
2          Dog Run  0.11
3   Scenic Lookout  0.11
4  Harbor / Marina  0.11


----Bay Colony (Fort Lauderdale, FL)----
                  venue  freq
0    Italian Restaurant   0.2
1     Convenience Store   0.1
2          Burger Joint   0.1
3  Fast Food Restaurant   0.1
4        Sandwich Place   0.1


----Bay Colony Club (Fort Lauderdale, FL)----
                  venue  freq
0          Burger Joint  0.05
1          

4       Automotive Shop  0.12


----Bellcamp Manor (West Little River, FL)----
                 venue  freq
0         Intersection   0.5
1  American Restaurant   0.5
2         Outlet Store   0.0
3          Pet Service   0.0
4  Peruvian Restaurant   0.0


----Belle Isle (Miami Beach, FL)----
                venue  freq
0               Hotel  0.12
1                Park  0.12
2  Italian Restaurant  0.06
3            Building  0.06
4                 Spa  0.06


----Bellevue Terrace (Sarasota, FL)----
                        venue  freq
0                 Yoga Studio  0.25
1              Lingerie Store  0.25
2  Construction & Landscaping  0.25
3                        Park  0.25
4                 Zoo Exhibit  0.00


----Bellview (Pensacola, FL)----
                 venue  freq
0       Discount Store   1.0
1          Zoo Exhibit   0.0
2         Outlet Store   0.0
3  Peruvian Restaurant   0.0
4   Persian Restaurant   0.0


----Belmar Gardens (Tampa, FL)----
                    venue  freq
0   

                 venue  freq
0   Athletics & Sports   0.5
1          Golf Course   0.5
2          Zoo Exhibit   0.0
3            Pet Store   0.0
4  Peruvian Restaurant   0.0


----Bloomingdale Hills (Riverview, FL)----
                  venue  freq
0  Fast Food Restaurant  0.18
1  Gym / Fitness Center  0.09
2            Nail Salon  0.09
3        Sandwich Place  0.09
4           Pizza Place  0.09


----Bloomingdale Ridge (Riverview, FL)----
            venue  freq
0    Burger Joint  0.12
1          Lounge  0.12
2      Playground  0.12
3  Shopping Plaza  0.12
4     Karaoke Bar  0.12


----Blues Creek (Gainesville, FL)----
                 venue  freq
0       Nightlife Spot  0.33
1       Adult Boutique  0.33
2                 Pool  0.33
3         Outlet Store  0.00
4  Peruvian Restaurant  0.00


----Bluffs (Jupiter, FL)----
                venue  freq
0                Pool   1.0
1         Zoo Exhibit   0.0
2         Outlet Mall   0.0
3  Persian Restaurant   0.0
4        Perfume Shop   0.0

...

4   Persian Restaurant   0.0


----Tigertown (Lakeland, FL)----
                 venue  freq
0       Baseball Field  0.67
1     Baseball Stadium  0.33
2          Zoo Exhibit  0.00
3            Nightclub  0.00
4  Peruvian Restaurant  0.00


----Tildenville (Winter Garden, FL)----
                 venue  freq
0           Food Truck   1.0
1          Outlet Mall   0.0
2  Peruvian Restaurant   0.0
3   Persian Restaurant   0.0
4         Perfume Shop   0.0


----Timacuan (Lake Mary, FL)----
                 venue  freq
0          Golf Course   1.0
1          Zoo Exhibit   0.0
2         Outlet Store   0.0
3  Peruvian Restaurant   0.0
4   Persian Restaurant   0.0


----Timber Lake Estates (Wesley Chapel, FL)----
                 venue  freq
0         Home Service   1.0
1          Zoo Exhibit   0.0
2  Peruvian Restaurant   0.0
3   Persian Restaurant   0.0
4         Perfume Shop   0.0


----Timber Pine (Riviera Beach, FL)----
            venue  freq
0      Donut Shop  0.12
1            Bank  0.12

                 venue  freq
0                Trail  0.33
1          Golf Course  0.33
2       Scenic Lookout  0.17
3  American Restaurant  0.17
4          Outlet Mall  0.00


----University Estates (Coral Gables, FL)----
                     venue  freq
0                 Pharmacy  0.12
1               Bagel Shop  0.12
2  Health & Beauty Service  0.12
3                      Gym  0.12
4   Furniture / Home Store  0.12


----University Park (Boca Raton, FL)----
                venue  freq
0  Golf Driving Range  0.25
1         Golf Course  0.25
2              Bistro  0.25
3                Pool  0.25
4         Zoo Exhibit  0.00


----University Park (Clearwater, FL)----
                        venue  freq
0  Construction & Landscaping  0.17
1             Automotive Shop  0.17
2           Other Repair Shop  0.17
3               Grocery Store  0.17
4               Big Box Store  0.17


----University Park (Jacksonville, FL)----
                 venue  freq
0          Flower Shop   1.0
1      

4          Outlet Store  0.00


----Washington Park (Hollywood, FL)----
                 venue  freq
0      Automotive Shop   1.0
1          Zoo Exhibit   0.0
2             Pharmacy   0.0
3          Pet Service   0.0
4  Peruvian Restaurant   0.0


----Washington Shores (Orlando, FL)----
                 venue  freq
0             Pharmacy  0.12
1            BBQ Joint  0.12
2                 Park  0.12
3        Shopping Mall  0.12
4  American Restaurant  0.12


----WaterSound Beach (Panama City Beach, FL)----
                 venue  freq
0  American Restaurant  0.25
1                Beach  0.25
2                  Gym  0.12
3       Ice Cream Shop  0.12
4            Beach Bar  0.12


----Waterford (Davie, FL)----
              venue  freq
0  Basketball Court  0.33
1              Food  0.33
2              Park  0.33
3       Zoo Exhibit  0.00
4       Outlet Mall  0.00


----Waterview (Miramar, FL)----
                 venue  freq
0       Farmers Market   0.5
1           Food Truck   0.5
2   

----Westchester (Miami, FL)----
                  venue  freq
0  Fast Food Restaurant  0.12
1         Grocery Store  0.07
2              Pharmacy  0.07
3        Discount Store  0.05
4                  Bank  0.05


----Westfield (Orlando, FL)----
               venue  freq
0       Intersection  0.17
1  Convenience Store  0.17
2         Toll Booth  0.17
3     Sandwich Place  0.17
4      Deli / Bodega  0.17


----Westfield (West Palm Beach, FL)----
            venue  freq
0  Sandwich Place   0.4
1       BBQ Joint   0.2
2            Food   0.2
3     Flea Market   0.2
4     Zoo Exhibit   0.0


----Westgate (Lakeland, FL)----
                 venue  freq
0       Baseball Field   1.0
1          Zoo Exhibit   0.0
2         Outlet Store   0.0
3  Peruvian Restaurant   0.0
4   Persian Restaurant   0.0


----Westhampton (Wellington, FL)----
            venue  freq
0  Nightlife Spot  0.33
1     Coffee Shop  0.33
2      Food Truck  0.33
3    Noodle House  0.00
4      Non-Profit  0.00


----Westhaven

                 venue  freq
0       Ice Cream Shop   0.5
1        Garden Center   0.5
2          Zoo Exhibit   0.0
3          Outlet Mall   0.0
4  Peruvian Restaurant   0.0


----Woodlake (Delray Beach, FL)----
                     venue  freq
0           Cosmetics Shop  0.25
1                Locksmith  0.25
2              Flower Shop  0.25
3  Comfort Food Restaurant  0.25
4              Zoo Exhibit  0.00


----Woodlake (Greenacres, FL)----
                  venue  freq
0         Grocery Store  0.33
1        Sandwich Place  0.33
2  Fast Food Restaurant  0.33
3           Zoo Exhibit  0.00
4          Outlet Store  0.00


----Woodland Acres (Jacksonville, FL)----
                 venue  freq
0  American Restaurant   1.0
1          Zoo Exhibit   0.0
2         Outlet Store   0.0
3          Pet Service   0.0
4  Peruvian Restaurant   0.0


----Woodland Estates (Jupiter, FL)----
                     venue  freq
0  Comfort Food Restaurant   0.5
1                      Spa   0.5
2              Z

In [38]:
def return_most_common_venues(row, num_top_venues):
    
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [39]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

columns = ['Neighborhood']

for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = candidates_grouped['Neighborhood']

for ind in np.arange(candidates_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(candidates_grouped.iloc[ind, :], num_top_venues)

#neighborhoods_venues_sorted.head()

## Cluster Neighborhoods

In [40]:
kclusters = 5

candidates_grouped_clustering = candidates_grouped.drop('Neighborhood', 1)

kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(candidates_grouped_clustering)

kmeans.labels_[0:10] 

array([0, 0, 0, 0, 0, 0, 0, 0, 3, 0])

In [41]:
neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"29th and Chase (Jacksonville, FL)",Sandwich Place,BBQ Joint,Chinese Restaurant,Discount Store,Zoo,Farmers Market,Empanada Restaurant,Erotic Museum,Ethiopian Restaurant,Event Service
1,"33rd St. Industrial (Orlando, FL)",Construction & Landscaping,Bakery,Toy / Game Store,Brewery,Skate Park,Gym,Lighting Store,General Entertainment,Gymnastics Gym,Event Service
2,"441 Corridor (Hollywood, FL)",Rental Car Location,Gas Station,Smoke Shop,Fondue Restaurant,Flower Shop,Electronics Store,Empanada Restaurant,Erotic Museum,Ethiopian Restaurant,Event Service
3,"5 Points (Sarasota, FL)",American Restaurant,Italian Restaurant,Coffee Shop,Breakfast Spot,Sushi Restaurant,Seafood Restaurant,Pizza Place,Gourmet Shop,Vegetarian / Vegan Restaurant,Lounge
4,"Aberdeen (Wesley Chapel, FL)",Gas Station,Yoga Studio,BBQ Joint,Construction & Landscaping,Home Service,Fabric Shop,Farmers Market,Farm,Falafel Restaurant,Factory


In [42]:
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

candidates_merged = location_data

candidates_merged = candidates_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='location')

candidates_merged.head()

,location,latitude,longitude,address,street,city,county,state,zipcode,male,...,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"CoreLife (833 North Krocks Rd, Suite 101)",40.563454,-75.562741,"5430 Us Hwy 222, Allentown, PA 18106",Us Hwy 222,Allentown,Lehigh County,PA,18106,0.430,...,Coffee Shop,Discount Store,Shoe Store,Grocery Store,Clothing Store,Bakery,Donut Shop,Gluten-free Restaurant,Gift Shop,Burger Joint
1,CoreLife (197 NW State Street),40.381365,-111.813989,"183 N West State Rd, American Fork, UT 84003",West State,American Fork,Utah County,UT,84003,0.528,...,Mexican Restaurant,Shoe Store,Department Store,Fast Food Restaurant,Burger Joint,Mobile Phone Shop,Sandwich Place,Clothing Store,Korean Restaurant,Laundry Service
2,CoreLife (1595 Niagara Falls Boulevard),43.000019,-78.822306,"1645 Niagara Falls Blvd, Buffalo, NY 14228",Niagara Falls,Buffalo,Erie County,NY,14228,0.528,...,Mobile Phone Shop,Coffee Shop,Clothing Store,Burger Joint,Mexican Restaurant,Salon / Barbershop,Pizza Place,Gift Shop,Steakhouse,Bakery
3,"CoreLife (205 North Maple Road, Suite 26)",42.282659,-83.780447,"2499 I- 94 Bus Lp, Ann Arbor, MI 48103",I- 94 Bus Lp,Ann Arbor,Washtenaw County,MI,48103,0.479,...,Coffee Shop,Pharmacy,Discount Store,American Restaurant,Department Store,Furniture / Home Store,Grocery Store,Salon / Barbershop,Sandwich Place,Baseball Field
4,CoreLife (700 Boardman-Poland Road),41.024610,-80.642790,"700 Boardman Poland Rd, Youngstown, OH 44512",Boardman Poland,Youngstown,Mahoning County,OH,44512,0.482,...,Furniture / Home Store,American Restaurant,Clothing Store,Sandwich Place,Kids Store,Cosmetics Shop,Mexican Restaurant,Mobile Phone Shop,Department Store,Shoe Store


In [43]:
candidates_merged.rename(columns={'latitude': 'Latitude', 'longitude': 'Longitude'}, inplace=True)

# Examine Clusters

In [44]:
candidates_merged.loc[candidates_merged['Cluster Labels'] == 0, candidates_merged.columns[[1] + list(range(5, candidates_merged.shape[1]))]]

,Latitude,city,county,state,zipcode,male,female,veterans,owners,renters,...,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,40.563454,Allentown,Lehigh County,PA,18106,0.430,0.570,0.064,0.866,0.134,...,Coffee Shop,Discount Store,Shoe Store,Grocery Store,Clothing Store,Bakery,Donut Shop,Gluten-free Restaurant,Gift Shop,Burger Joint
1,40.381365,American Fork,Utah County,UT,84003,0.528,0.472,0.068,0.514,0.486,...,Mexican Restaurant,Shoe Store,Department Store,Fast Food Restaurant,Burger Joint,Mobile Phone Shop,Sandwich Place,Clothing Store,Korean Restaurant,Laundry Service
2,43.000019,Buffalo,Erie County,NY,14228,0.528,0.472,0.041,0.198,0.802,...,Mobile Phone Shop,Coffee Shop,Clothing Store,Burger Joint,Mexican Restaurant,Salon / Barbershop,Pizza Place,Gift Shop,Steakhouse,Bakery
3,42.282659,Ann Arbor,Washtenaw County,MI,48103,0.479,0.521,0.045,0.532,0.468,...,Coffee Shop,Pharmacy,Discount Store,American Restaurant,Department Store,Furniture / Home Store,Grocery Store,Salon / Barbershop,Sandwich Place,Baseball Field
4,41.024610,Youngstown,Mahoning County,OH,44512,0.482,0.518,0.067,0.770,0.230,...,Furniture / Home Store,American Restaurant,Clothing Store,Sandwich Place,Kids Store,Cosmetics Shop,Mexican Restaurant,Mobile Phone Shop,Department Store,Shoe Store
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2562,28.312681,Kissimmee,Osceola County,FL,34747,0.471,0.529,0.079,0.618,0.382,...,Gym,Resort,Pharmacy,Building,Farmers Market,Electronics Store,Empanada Restaurant,Erotic Museum,Ethiopian Restaurant,Event Service
2563,30.441848,Shalimar,Okaloosa County,FL,32579,0.480,0.520,0.176,0.506,0.494,...,Convenience Store,Baseball Field,Zoo,Field,Erotic Museum,Ethiopian Restaurant,Event Service,Event Space,Exhibit,Fabric Shop
2565,27.781464,Saint Petersburg,Pinellas County,FL,33706,0.471,0.529,0.109,0.957,0.043,...,Baseball Field,Boat or Ferry,Seafood Restaurant,Zoo,Fast Food Restaurant,Erotic Museum,Ethiopian Restaurant,Event Service,Event Space,Exhibit
2570,28.996888,Altoona,Lake County,FL,32702,0.389,0.611,0.152,0.568,0.432,...,Food Truck,Field,Empanada Restaurant,Erotic Museum,Ethiopian Restaurant,Event Service,Event Space,Exhibit,Fabric Shop,Factory


In [45]:
candidates_merged.describe()

,Latitude,Longitude,zipcode,male,female,veterans,owners,renters,median_home,families,median_income,median_age,hispanic,non_hispanic,Cluster Labels
count,2575.000000,2575.000000,2575.000000,2575.000000,2575.000000,2575.000000,2575.000000,2575.000000,2.575000e+03,2575.000000,2575.000000,2575.000000,2575.000000,2575.000000,2242.000000
mean,27.858484,-81.506866,33418.729709,0.476666,0.523336,0.085115,0.671328,0.328673,2.456369e+05,0.644404,62807.135146,43.333515,0.206464,0.793537,0.300178
std,2.405747,1.912336,3233.030545,0.058539,0.058540,0.054091,0.216432,0.216432,2.177195e+05,0.163173,32306.193125,10.578097,0.221639,0.221639,0.872245
min,24.549922,-111.890963,11530.000000,0.234000,0.120000,0.000000,0.058000,0.000000,9.999000e+03,0.075000,12742.000000,19.900000,0.000000,0.000000,0.000000
25%,26.340840,-82.476776,33018.000000,0.444000,0.491000,0.043000,0.523000,0.157000,1.267000e+05,0.538000,40052.000000,35.600000,0.056000,0.728500,0.000000
50%,27.759134,-81.461244,33414.000000,0.475000,0.525000,0.079000,0.709000,0.291000,1.881000e+05,0.660000,56414.000000,41.800000,0.128000,0.872000,0.000000
75%,28.360769,-80.235149,33759.000000,0.509000,0.556000,0.119000,0.843000,0.477000,2.920500e+05,0.765500,77553.000000,49.250000,0.271500,0.944000,0.000000
max,43.213752,-73.419049,84111.000000,0.880000,0.766000,0.341000,1.000000,0.942000,2.000001e+06,1.000000,250001.000000,88.400000,1.000000,1.000000,4.000000


## Save DataFrames

In [46]:
df = pd.DataFrame(candidates_merged)
gdf = gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(df.Longitude, df.Latitude))
gdf.dropna(inplace=True)
gdf = gdf.reset_index().drop(columns=['index', 
                                      '1st Most Common Venue', '2nd Most Common Venue', '3rd Most Common Venue', 
                                      '4th Most Common Venue', '5th Most Common Venue', '6th Most Common Venue', 
                                      '7th Most Common Venue', '8th Most Common Venue', '9th Most Common Venue', 
                                      '10th Most Common Venue'])
gdf.to_file(driver = 'ESRI Shapefile', filename='CoreLife_Clusters.shp')

In [47]:
gdf

,location,Latitude,Longitude,address,street,city,county,state,zipcode,male,...,owners,renters,median_home,families,median_income,median_age,hispanic,non_hispanic,Cluster Labels,geometry
0,"CoreLife (833 North Krocks Rd, Suite 101)",40.563454,-75.562741,"5430 Us Hwy 222, Allentown, PA 18106",Us Hwy 222,Allentown,Lehigh County,PA,18106,0.430,...,0.866,0.134,255600,0.786,74355,38.4,0.082,0.918,0.0,POINT (-75.56274 40.56345)
1,CoreLife (197 NW State Street),40.381365,-111.813989,"183 N West State Rd, American Fork, UT 84003",West State,American Fork,Utah County,UT,84003,0.528,...,0.514,0.486,138000,0.734,42391,29.3,0.168,0.832,0.0,POINT (-111.81399 40.38137)
2,CoreLife (1595 Niagara Falls Boulevard),43.000019,-78.822306,"1645 Niagara Falls Blvd, Buffalo, NY 14228",Niagara Falls,Buffalo,Erie County,NY,14228,0.528,...,0.198,0.802,85500,0.374,16005,33.3,0.033,0.967,0.0,POINT (-78.82231 43.00002)
3,"CoreLife (205 North Maple Road, Suite 26)",42.282659,-83.780447,"2499 I- 94 Bus Lp, Ann Arbor, MI 48103",I- 94 Bus Lp,Ann Arbor,Washtenaw County,MI,48103,0.479,...,0.532,0.468,222000,0.542,77539,38.4,0.045,0.955,0.0,POINT (-83.78045 42.28266)
4,CoreLife (700 Boardman-Poland Road),41.024610,-80.642790,"700 Boardman Poland Rd, Youngstown, OH 44512",Boardman Poland,Youngstown,Mahoning County,OH,44512,0.482,...,0.770,0.230,121900,0.559,60242,48.1,0.018,0.982,0.0,POINT (-80.64279 41.02461)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2237,"Lake Lorraine (Shalimar, FL)",30.441848,-86.566234,"44 10th St, Shalimar, FL 32579",10th,Shalimar,Okaloosa County,FL,32579,0.480,...,0.506,0.494,158200,0.600,56271,36.3,0.188,0.812,0.0,POINT (-86.56623 30.44185)
2238,"Paradise Island (Treasure Island, FL)",27.765084,-82.759306,"10225 Paradise Blvd, Saint Petersburg, FL 33706",Paradise,Saint Petersburg,Pinellas County,FL,33706,0.477,...,0.817,0.183,309200,0.521,78553,62.8,0.082,0.918,3.0,POINT (-82.75931 27.76508)
2239,"Isle of Capri (Treasure Island, FL)",27.781464,-82.770970,"460 Capri Blvd, Saint Petersburg, FL 33706",Capri,Saint Petersburg,Pinellas County,FL,33706,0.471,...,0.957,0.043,613200,0.680,82500,59.0,0.091,0.909,0.0,POINT (-82.77097 27.78146)
2240,"Pittman (Altoona, FL)",28.996888,-81.644155,"19005 Lake Rd, Altoona, FL 32702",Lake,Altoona,Lake County,FL,32702,0.389,...,0.568,0.432,112500,0.512,37950,61.2,0.068,0.932,0.0,POINT (-81.64415 28.99689)
